# Protein

lamindb provides access to the following public protein ontologies through [lnschema-bionty](https://github.com/laminlabs/lnschema-bionty):

1. [Uniprot](https://www.uniprot.org/)

Here we show how to access and search Protein ontologies to standardize new data.

## Setup

In [ ]:
!lamin init --storage ./test-protein --schema bionty

In [ ]:
import lnschema_bionty as lb
import pandas as pd

## PublicOntology objects

Let us create a public ontology accessor with {meth}`~docs:lnschema_bionty.dev.BioRegistry.public`, which chooses a default public ontology source from {class}`~docs:lnschema_bionty.PublicSource`.
It's a [PublicOntology](https://lamin.ai/docs/lnschema_bionty.dev.publicontology) object, which you can think about as a public registry:

In [ ]:
proteins = lb.Protein.public(organism="human")
proteins

As for registries, you can export the ontology as a `DataFrame`:

In [ ]:
df = proteins.df()
df.head()

Unlike registries, you can also export it as a Pronto object via `public.ontology`.

## Look up terms

As for registries, terms can be looked up with auto-complete:

In [ ]:
lookup = proteins.lookup()

The `.` accessor provides normalized terms (lower case, only contains alphanumeric characters and underscores):

In [ ]:
lookup.ac3

To look up the exact original strings, convert the lookup object to dict and use the `[]` accessor:

In [ ]:
lookup_dict = lookup.dict()
lookup_dict["AC3"]

By default, the `name` field is used to generate lookup keys. You can specify another field to look up:

In [ ]:
lookup = proteins.lookup(proteins.gene_symbol)

In [ ]:
lookup.rab4a

## Search terms

Search behaves in the same way as it does for registries:

In [ ]:
proteins.search("RAS").head(3)

By default, search also covers synonyms:

In [ ]:
proteins.search("member of RAS oncogene family like 2B").head(3)

You can turn this off synonym by passing `synonyms_field=None`:

In [ ]:
proteins.search("member of RAS oncogene family like 2B", synonyms_field=None).head(3)

Search another field (default is `.name`):

In [ ]:
proteins.search(
    "RABL2B",
    field=proteins.definition,
).head()

## Standardize Protein identifiers

Let us generate a `DataFrame` that stores a number of Protein identifiers, some of which corrupted:

In [ ]:
df_orig = pd.DataFrame(
    index=[
        "A0A024QZ08"
        "X6RLV5"
        "X6RM24"
        "A0A024QZQ1"
        "This protein does not exist",
    ]
)
df_orig

We can check whether any of our values are validated against the ontology reference:

In [ ]:
validated = proteins.validate(df_orig.index, proteins.name)
df_orig.index[~validated]

## Ontology source versions

For any given entity, we can choose from a number of versions:

In [ ]:
lb.PublicSource.filter(entity="Protein").df()

When instantiating a Bionty object, we can choose a source or version:

In [ ]:
public_source = lb.PublicSource.filter(
    source="uniprot", version="2023-03", organism="human"
).one()
proteins= lb.Protein.public(public_source=public_source)
proteins

The currently used ontologies can be displayed using:

In [ ]:
lb.PublicSource.filter(currently_used=True).df()

In [ ]:
!lamin delete --force test-protein
!rm -r test-protein